# Getting Anilist users
* Turns an anilist userid into a username

In [ ]:
import contextlib
import datetime
import glob
import json
import logging
import os
import random
import shutil
import time

import numpy as np
import pandas as pd
import requests
from ratelimit import limits, sleep_and_retry
from tqdm import tqdm

In [ ]:
name = "anilist"

In [ ]:
# outdir
data_path = f"../../data/{name}/user_facts"
if not os.path.exists(data_path):
    os.makedirs(data_path)
os.chdir(data_path)

In [ ]:
# logging
logger = logging.getLogger("GetUsers")
logger.setLevel(logging.DEBUG)
formatter = logging.Formatter(
    "%(name)s:%(levelname)s:%(asctime)s: %(message)s", datefmt="%Y-%m-%d %H:%M:%S"
)
for stream in [
    logging.FileHandler("get_users.log"),
    logging.StreamHandler(),
]:
    stream.setFormatter(formatter)
    logger.addHandler(stream)

## Parse AniList API

In [ ]:
@sleep_and_retry
@limits(calls=1, period=1)
def call_api(url, json, retry_timeout=1):
    try:
        response = requests.post(url, json=json)
        if response.status_code in [429, 500, 504, 522] and retry_timeout < 3600:
            # This can occur if servers go down or if the page doesnt exist
            raise Exception(f"{response.status_code}")
    except Exception as e:
        logger.warning(
            f"Received error {str(e)} while accessing {url}. Retrying in {retry_timeout} seconds"
        )
        time.sleep(retry_timeout)
        retry_timeout = min(retry_timeout * 2, 3600)
        return call_api(url, json, retry_timeout)
    return response

In [ ]:
def get_username(userid):
    url = "https://graphql.anilist.co"
    query = "query ($userid: Int) { User (id: $userid) { name } }"
    variables = {"userid": str(userid)}
    response = call_api(url, {"query": query, "variables": variables})
    try:
        response.raise_for_status()
    except Exception as e:
        logger.warning(f"Received error {str(e)} while accessing {url}")
        return f"{userid}"
    return response.json()["data"]["User"]["name"]

In [ ]:
def get_userid(username):
    url = "https://graphql.anilist.co"
    query = "query ($name: String) { User (name: $name) { id } }"
    variables = {"name": str(username)}
    response = call_api(url, {"query": query, "variables": variables})
    try:
        response.raise_for_status()
    except Exception as e:
        logger.warning(f"Received error {str(e)} while accessing {url}")
        return f"{userid}"
    return response.json()["data"]["User"]["id"]

In [ ]:
# atomic saving utilities
@contextlib.contextmanager
def atomic_append(filename):
    temp = filename + "~"
    with open(temp, "w") as f:
        yield f

    temp2 = temp + "~"
    with open(temp2, "wb") as wfd:
        for f in [filename, temp]:
            with open(f, "rb") as fd:
                shutil.copyfileobj(fd, wfd)
    os.remove(temp)
    os.replace(temp2, filename)


def atomic_append_dataframe_to_csv(df, filename):
    first_run = not os.path.exists(filename)
    if first_run:
        with open(filename, "w") as fp:
            pass
    temp = filename + "~"
    with atomic_append(filename) as f:
        df.to_csv(f, index=False, header=first_run)

In [ ]:
# snapshot hourly to amortize the cost of the disk I/O
def should_save(reason):
    should_save = False
    if reason not in save_reasons:
        save_reasons[reason] = (0, 1)
    iterations_since_last_write, iterations_until_next_write = save_reasons[reason]
    iterations_since_last_write += 1
    if iterations_since_last_write >= iterations_until_next_write:
        iterations_since_last_write = 0
        iterations_until_next_write = min(2 * iterations_until_next_write, 5400)
        should_save = True
        logger.info(
            f"Writing data for {reason}. Will next write data "
            f"after {iterations_until_next_write} iterations"
        )
    save_reasons[reason] = (iterations_since_last_write, iterations_until_next_write)
    return should_save


save_reasons = {}

In [ ]:
files = glob.glob("../user_anime_facts/user_status.*.csv")
user_df = pd.concat(pd.read_csv(f) for f in files)
user_df = user_df.loc[lambda x: x["failed_attempts"] == 0]

In [ ]:
outfile = "userid_to_username.csv"
if os.path.exists(outfile):
    existing_userids = list(pd.read_csv(outfile)["userid"])
else:
    existing_userids = []
new_userids = list(set(user_df.username) - set(existing_userids))
random.shuffle(new_userids)
num_userids = len(existing_userids)

In [ ]:
logger.info(
    f"Getting usernames for {len(new_userids)} new userids and {len(existing_userids)} existing userids"
)

In [ ]:
out_userids = []
out_usernames = []
for userid in new_userids:
    username = get_username(userid)
    out_userids.append(userid)
    out_usernames.append(username)
    if should_save("users"):
        atomic_append_dataframe_to_csv(
            pd.DataFrame.from_dict({"userid": out_userids, "username": out_usernames}),
            outfile,
        )
        num_userids += len(out_usernames)
        logger.info(
            f"Successfully written {num_userids} usernames. "
            f"{len(new_userids) - len(out_userids)} usernames remaining"
        )
        out_userids = []
        out_usernames = []

In [ ]:
atomic_append_dataframe_to_csv(
    pd.DataFrame.from_dict({"userid": out_userids, "username": out_usernames}),
    outfile,
)
num_userids += len(out_usernames)
logger.info(f"Successfully written {num_userids} usernames")
logger.info("Finished writing all usernames!")